In [1]:
import requests
from bs4 import BeautifulSoup
import yfinance as yf
from datetime import datetime, date

from datetime import timedelta
import pandas as pd
import json
import numpy as np
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
import time
import re
import dateutil.parser
from functions import (get_sector, changes_from_press, get_content_header, 
                       get_df, normalise_to_index, save_data)
final_date = date(2014, 1, 1)
past_last_date = False

C:\Users\Aidan\miniconda3\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
year = datetime.today().year

driver = webdriver.Firefox()
driver.get('https://investor.spglobal.com/news-releases/default.aspx')

data = []

ticker = 'spgi'
sector = get_sector(ticker)
index_ticker = '^GSPC'
 

stock_hist = get_df(ticker)
index_hist = get_df(index_ticker)

year = datetime.today().year

while True:

    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html) 

    articles = soup.find_all('div',attrs={'class':'irwTableRowItem'})
    
    for article in articles:
        date = article.find('div', attrs={'class':'irwPRDate'}).text.lstrip().rstrip() 
        
        title = article.find('a').text.rstrip() 
        url = article.find('a').get('href')
        url = f'https://investor.spglobal.com{url}'
        
        date = dateutil.parser.parse(date).date()
        if date < final_date:
            past_last_date = True
            break
        content = get_content_header(url, 'meta', {'name':'twitter:description'})
        pct_change = changes_from_press(stock_hist, date, 1)
        
        if pct_change != None:
            index_price, norm_price = normalise_to_index(pct_change, date, index_hist)
        else:
            index_price, norm_price = (None, None)

        data.append([str(date), title, url, content, pct_change, index_price, norm_price])
    if past_last_date == True:
        break
    year = year - 1
    

driver.close()

save_data(data, ticker, sector)